# Dash Callbacks

Steps to create a callback for interactions:
* Create a function to return some desired output.
* Decorate that function with an **@app.callback** decorator.
    * Set an Output to a component id.
    * Set an Input to a component id.
* Connect the desired properties.

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output 
import pandas as pd
import plotly.graph_objs as go

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    dcc.Input(id='my-id', value='Initial Text', type='text'),
    html.Div(id='my-div', style={'border':'2px blue solid'})
])


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


### Connecting through a callback

**Adding a callback to one component**


For this exercise we’ll add a callback to an input box, and display the data being entered as an immediate output on the same screen.

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    dcc.Input(id='my-id', value='Initial Text', type='text'),
    html.Div(id='my-div', style={'border':'2px blue solid'})
])


@app.callback(Output(component_id='my-div', component_property='children'),
             [Input(component_id='my-id', component_property='value')])
def update_output_div(input_value):
    return(f"You entered: {input_value}")




if __name__ == '__main__':
    app.run_server()
    
    
    
    

Let's break down what's happening here:

1. We set up our dcc.Input in the usual way, except that we assigned an id to it, and added another Div after it with an assigned id ('my-id' and 'my-div' respectively)
2. app.callback is called as a decorator function over update_output_div. The "inputs" and "outputs" of our application interface are described declaratively through the app.callback decorator.
    For more on Python decorators visit https://en.wikipedia.org/wiki/Python_syntax_and_semantics#Decorators 
3. Inside @app.callback, Output and Input are abbreviated forms of dash.dependencies.Output and dash.dependencies.Input. Note how we imported them from dash.dependencies by name.
4. In Dash, the inputs and outputs of our application are simply the properties of a particular component. 
    In this example, our input is the "value" property of the component that has the ID "my-id". 
    Our output is the "children" property of the component with the ID "my-div".
5. Whenever an input property changes, the function that the callback decorator wraps will get called automatically. Dash provides the function with the new value of the input property as an input argument and Dash updates the property of the output component with whatever was returned by the function.
6. The component_id and component_property keywords inside Output and Input are optional (there are only two arguments for each of those objects). We included them here for clarity but we’ll omit them from here on out for brevity and readability.
7. Don't confuse the dash.dependencies.Input object inside app.callback from the dash_core_components.Input object inside app.layout. The former is just used in these callbacks and the latter is an actual component.
8. Notice how we don't set a value for the children property of the my-div component in the layout. When the Dash app starts, it automatically calls all of the callbacks with the initial values of the input components in order to populate the initial state of the output components. In this example, if you specified something like html.Div(id='my-div', children='Hello world'), it would get overwritten when the app starts.


It's sort of like programming with Microsoft Excel: whenever an input cell changes, all of the cells that depend on that cell will get updated automatically. This is called "Reactive Programming".
Remember how every component was described entirely through its set of keyword arguments? Those properties are important now. With Dash interactivity, we can dynamically update any of those properties through a callback function. Frequently we'll update the children of an html component to display new text or the figure of a dcc.Graph component to display new data, but we could also update the style of a component or even the available options of a dcc.Dropdown component!


### Connecting two components with callbacks

The next example comes from the Dash online tutorial, and it’s fairly complex. It introduces some layout features we haven’t seen before, like a logarithmic x-axis. The goal is to have an interactive Slider update a Graph on the same page. We will use a Dash dataset available online at https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv 



In [ ]:
df = pd.read_csv('gapminderDataFiveYear.csv')
 
app = dash.Dash()
 
 
# https://dash.plot.ly/dash-core-components/dropdown
# We need to construct a dictionary of dropdown values for the years
year_options = []
for year in df['year'].unique():
    year_options.append({'label':str(year),'value':year})
 
app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Dropdown(id='year-picker',options=year_options,value=df['year'].min())
])
 
@app.callback(Output('graph-with-slider', 'figure'),
              [Input('year-picker', 'value')])
def update_figure(selected_year):
    filtered_df = df[df['year'] == selected_year]
    traces = []
    for continent_name in filtered_df['continent'].unique():
        df_by_continent = filtered_df[filtered_df['continent'] == continent_name]
        traces.append(go.Scatter(
            x=df_by_continent['gdpPercap'],
            y=df_by_continent['lifeExp'],
            text=df_by_continent['country'],
            mode='markers',
            opacity=0.7,
            marker={'size': 15},
            name=continent_name
        ))
 
    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'type': 'log', 'title': 'GDP Per Capita'},
            yaxis={'title': 'Life Expectancy'},
            hovermode='closest'
        )
    }
 
if __name__ == '__main__':
    app.run_server()


## Dash callbacks for graphs

In [2]:
df = pd.read_csv('gapminderDataFiveYear.csv')
print(df.head())

       country  year         pop continent  lifeExp   gdpPercap
0  Afghanistan  1952   8425333.0      Asia   28.801  779.445314
1  Afghanistan  1957   9240934.0      Asia   30.332  820.853030
2  Afghanistan  1962  10267083.0      Asia   31.997  853.100710
3  Afghanistan  1967  11537966.0      Asia   34.020  836.197138
4  Afghanistan  1972  13079460.0      Asia   36.088  739.981106


In [ ]:
app = dash.Dash()

year_options = []

for year in df['year'].unique():
    year_options.append({'label': str(year), 'value': year})


app.layout = html.Div([
    dcc.Graph(id='graph'),
    dcc.Dropdown(id='year_picker', options=year_options,
                value=df['year'].min())
])